# data 

In [1]:
import torch 
import importlib
import monotonic
import data_manager
import metrics
import utils
importlib.reload(utils)
import train as Train
from train import execute_model_evaluation
import train_config
from data_manager import DatasetManager
from train_config import FlatACLConfig, FlatDLinearConfig, FlatNaiveConfig, FlatPatchTSTConfig, FlatTimeMixerConfig
from dataclasses import replace

%load_ext autoreload
%autoreload 2
modules_to_reload_list = [
    data_manager,
    Train,
    train_config,
    monotonic,
    # data_manager, # Reloaded only once even if listed twice
    utils,
    # train_config, # Reloaded only once even if listed twice
    metrics
]

# Initialize the data manager
data_mgr = DatasetManager(device='cuda')

# Load a synthetic dataset
data_mgr.load_trajectory('rossler', steps=51999, dt=1e-2) # 5039

RosslerSystem initialized with method: rk4 on device: cuda

Dataset: rossler (synthetic)
Shape: torch.Size([52000, 3])
Channels: 3
Length: 52000
Parameters: {'steps': 51999, 'dt': 0.01}

Sample data (first 2 rows):
tensor([[1.0000, 1.0000, 1.0000],
        [0.9802, 1.0119, 0.9559]], device='cuda:0')


## Seq=336

### EigenACL

#### pred=96

##### huber

In [12]:
from monotonic import DynamicTanh
import torch.nn as nn

importlib.reload(monotonic)
importlib.reload(train_config) 
cfg = train_config.FlatACLConfig(  # original householder 
    seq_len=336,
    pred_len=96,
    channels=data_mgr.datasets['rossler']['channels'],# data_mgr.channels,              # ← number of features in your data
    batch_size=128,
    learning_rate=9e-4, 
    seeds=[1955, 7, 20],  
    epochs=50, 
    dim_hidden=128,
    dim_augment=128, 
    ablate_no_koopman=False,
    use_complex_eigenvalues=True,
    second_delay_use_shift=True,
    ablate_rotate_back_Koopman=True, 
    ablate_shift_inside_scale=False,
    householder_reflects_latent = 2,
    householder_reflects_data = 4,
    mixing_strategy='delay_only', 
    loss_backward_weights = [0.0, 0.0, 1.0, 0.0, 0.0],
    loss_validate_weights = [0.0, 0.0, 1.0, 0.0, 0.0],
    ablate_deterministic_y0=False, 
)
cfg.x_to_z_delay.enable_magnitudes = [False, True]
cfg.x_to_z_delay.spectral_flags_scale_shift = [True, False]
cfg.x_to_z_delay.spectral_flags_magnitudes = [False, True]
cfg.x_to_z_delay.spectral_flags_hidden_layers = [False, False]
cfg.x_to_z_delay.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.x_to_z_delay.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]

cfg.x_to_z_deri.enable_magnitudes = [False, True]
cfg.x_to_z_deri.spectral_flags_scale_shift = [True, False]
cfg.x_to_z_deri.spectral_flags_magnitudes = [False, True]
cfg.x_to_z_deri.spectral_flags_hidden_layers = [False, False]
cfg.x_to_z_deri.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.x_to_z_deri.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]

cfg.z_to_x_main.enable_magnitudes = [False, True]
cfg.z_to_x_main.spectral_flags_scale_shift = [True, False]
cfg.z_to_x_main.spectral_flags_magnitudes = [False, True]
cfg.z_to_x_main.spectral_flags_hidden_layers = [False, False]
cfg.z_to_x_main.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_to_x_main.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]

cfg.z_push_to_z.enable_magnitudes = [False, True]
cfg.z_push_to_z.spectral_flags_scale_shift = [True, False]
cfg.z_push_to_z.spectral_flags_magnitudes = [False, True]
cfg.z_push_to_z.spectral_flags_hidden_layers = [False, False]
cfg.z_push_to_z.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_push_to_z.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]

cfg.z_to_y_main.enable_magnitudes = [False, True]
cfg.z_to_y_main.spectral_flags_scale_shift = [True, False]
cfg.z_to_y_main.spectral_flags_magnitudes = [False, True]
cfg.z_to_y_main.spectral_flags_hidden_layers = [False, False]
cfg.z_to_y_main.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_to_y_main.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]
exp = execute_model_evaluation('rossler', cfg, data_mgr, scale=False)

Shape of training data: torch.Size([36400, 3])
Shape of validation data: torch.Size([5200, 3])
Shape of testing data: torch.Size([10400, 3])
global_std.shape: torch.Size([3])
Global Std for rossler: tensor([5.0964, 4.7939, 2.8266], device='cuda:0')
Train set sample shapes: torch.Size([336, 3]), torch.Size([96, 3])
Validation set sample shapes: torch.Size([336, 3]), torch.Size([96, 3])
Test set data shapes: torch.Size([10400, 3]), torch.Size([10400, 3])
Number of batches in train_loader: 282
Batch 0: Data shape torch.Size([128, 336, 3]), Target shape torch.Size([128, 96, 3])

Data Preparation: rossler
Sequence Length: 336
Prediction Length: 96
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 282
Validation Batches: 38
Test Batches: 78

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 2.1727, mae: 0.5053, huber: 0.3101, swd: 1.8004, ept: 91.4372
Epoch [1/50], Val Losses: mse: 0.9068, 

#### pred=196

##### huber

In [13]:
from monotonic import DynamicTanh
import torch.nn as nn

importlib.reload(monotonic)
importlib.reload(train_config) 
cfg = train_config.FlatACLConfig(  # original householder 
    seq_len=336,
    pred_len=196,
    channels=data_mgr.datasets['rossler']['channels'],# data_mgr.channels,              # ← number of features in your data
    batch_size=128,
    learning_rate=9e-4, 
    seeds=[1955, 7, 20],  
    epochs=50, 
    dim_hidden=128,
    dim_augment=128, 
    ablate_no_koopman=False,
    use_complex_eigenvalues=True,
    second_delay_use_shift=True,
    ablate_rotate_back_Koopman=True, 
    ablate_shift_inside_scale=False,
    householder_reflects_latent = 2,
    householder_reflects_data = 4,
    mixing_strategy='delay_only', 
    loss_backward_weights = [0.0, 0.0, 1.0, 0.0, 0.0],
    loss_validate_weights = [0.0, 0.0, 1.0, 0.0, 0.0],
    ablate_deterministic_y0=False, 
)
cfg.x_to_z_delay.enable_magnitudes = [False, True]
cfg.x_to_z_delay.spectral_flags_scale_shift = [True, False]
cfg.x_to_z_delay.spectral_flags_magnitudes = [False, True]
cfg.x_to_z_delay.spectral_flags_hidden_layers = [False, False]
cfg.x_to_z_delay.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.x_to_z_delay.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]

cfg.x_to_z_deri.enable_magnitudes = [False, True]
cfg.x_to_z_deri.spectral_flags_scale_shift = [True, False]
cfg.x_to_z_deri.spectral_flags_magnitudes = [False, True]
cfg.x_to_z_deri.spectral_flags_hidden_layers = [False, False]
cfg.x_to_z_deri.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.x_to_z_deri.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]

cfg.z_to_x_main.enable_magnitudes = [False, True]
cfg.z_to_x_main.spectral_flags_scale_shift = [True, False]
cfg.z_to_x_main.spectral_flags_magnitudes = [False, True]
cfg.z_to_x_main.spectral_flags_hidden_layers = [False, False]
cfg.z_to_x_main.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_to_x_main.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]

cfg.z_push_to_z.enable_magnitudes = [False, True]
cfg.z_push_to_z.spectral_flags_scale_shift = [True, False]
cfg.z_push_to_z.spectral_flags_magnitudes = [False, True]
cfg.z_push_to_z.spectral_flags_hidden_layers = [False, False]
cfg.z_push_to_z.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_push_to_z.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]

cfg.z_to_y_main.enable_magnitudes = [False, True]
cfg.z_to_y_main.spectral_flags_scale_shift = [True, False]
cfg.z_to_y_main.spectral_flags_magnitudes = [False, True]
cfg.z_to_y_main.spectral_flags_hidden_layers = [False, False]
cfg.z_to_y_main.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_to_y_main.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]
exp = execute_model_evaluation('rossler', cfg, data_mgr, scale=False)

Shape of training data: torch.Size([36400, 3])
Shape of validation data: torch.Size([5200, 3])
Shape of testing data: torch.Size([10400, 3])
global_std.shape: torch.Size([3])
Global Std for rossler: tensor([5.0964, 4.7939, 2.8266], device='cuda:0')
Train set sample shapes: torch.Size([336, 3]), torch.Size([196, 3])
Validation set sample shapes: torch.Size([336, 3]), torch.Size([196, 3])
Test set data shapes: torch.Size([10400, 3]), torch.Size([10400, 3])
Number of batches in train_loader: 281
Batch 0: Data shape torch.Size([128, 336, 3]), Target shape torch.Size([128, 196, 3])

Data Preparation: rossler
Sequence Length: 336
Prediction Length: 196
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 281
Validation Batches: 37
Test Batches: 78

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 3.2397, mae: 0.6173, huber: 0.4033, swd: 2.9480, ept: 180.3137
Epoch [1/50], Val Losses: mse: 2.2

#### pred=336

##### huber

In [2]:
from monotonic import DynamicTanh
import torch.nn as nn

importlib.reload(monotonic)
importlib.reload(train_config) 
cfg = train_config.FlatACLConfig(  # original householder 
    seq_len=336,
    pred_len=336,
    channels=data_mgr.datasets['rossler']['channels'],# data_mgr.channels,              # ← number of features in your data
    batch_size=128,
    learning_rate=9e-4, 
    seeds=[1955, 7, 20],  
    epochs=50, 
    dim_hidden=128,
    dim_augment=128, 
    ablate_no_koopman=False,
    use_complex_eigenvalues=True,
    second_delay_use_shift=True,
    ablate_rotate_back_Koopman=True, 
    ablate_shift_inside_scale=False,
    householder_reflects_latent = 2,
    householder_reflects_data = 4,
    mixing_strategy='delay_only', 
    loss_backward_weights = [0.0, 0.0, 1.0, 0.0, 0.0],
    loss_validate_weights = [0.0, 0.0, 1.0, 0.0, 0.0],
    ablate_deterministic_y0=False, 
)
cfg.x_to_z_delay.enable_magnitudes = [False, True]
cfg.x_to_z_delay.spectral_flags_scale_shift = [True, False]
cfg.x_to_z_delay.spectral_flags_magnitudes = [False, True]
cfg.x_to_z_delay.spectral_flags_hidden_layers = [False, False]
cfg.x_to_z_delay.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.x_to_z_delay.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]

cfg.x_to_z_deri.enable_magnitudes = [False, True]
cfg.x_to_z_deri.spectral_flags_scale_shift = [True, False]
cfg.x_to_z_deri.spectral_flags_magnitudes = [False, True]
cfg.x_to_z_deri.spectral_flags_hidden_layers = [False, False]
cfg.x_to_z_deri.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.x_to_z_deri.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]

cfg.z_to_x_main.enable_magnitudes = [False, True]
cfg.z_to_x_main.spectral_flags_scale_shift = [True, False]
cfg.z_to_x_main.spectral_flags_magnitudes = [False, True]
cfg.z_to_x_main.spectral_flags_hidden_layers = [False, False]
cfg.z_to_x_main.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_to_x_main.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]

cfg.z_push_to_z.enable_magnitudes = [False, True]
cfg.z_push_to_z.spectral_flags_scale_shift = [True, False]
cfg.z_push_to_z.spectral_flags_magnitudes = [False, True]
cfg.z_push_to_z.spectral_flags_hidden_layers = [False, False]
cfg.z_push_to_z.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_push_to_z.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]

cfg.z_to_y_main.enable_magnitudes = [False, True]
cfg.z_to_y_main.spectral_flags_scale_shift = [True, False]
cfg.z_to_y_main.spectral_flags_magnitudes = [False, True]
cfg.z_to_y_main.spectral_flags_hidden_layers = [False, False]
cfg.z_to_y_main.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_to_y_main.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]
exp = execute_model_evaluation('rossler', cfg, data_mgr, scale=False)

Shape of training data: torch.Size([36400, 3])
Shape of validation data: torch.Size([5200, 3])
Shape of testing data: torch.Size([10400, 3])
global_std.shape: torch.Size([3])
Global Std for rossler: tensor([5.0964, 4.7939, 2.8266], device='cuda:0')
Train set sample shapes: torch.Size([336, 3]), torch.Size([336, 3])
Validation set sample shapes: torch.Size([336, 3]), torch.Size([336, 3])
Test set data shapes: torch.Size([10400, 3]), torch.Size([10400, 3])
Number of batches in train_loader: 280
Batch 0: Data shape torch.Size([128, 336, 3]), Target shape torch.Size([128, 336, 3])

Data Preparation: rossler
Sequence Length: 336
Prediction Length: 336
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 280
Validation Batches: 36
Test Batches: 77

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 3.6815, mae: 0.6937, huber: 0.4643, swd: 3.0216, ept: 296.5612
Epoch [1/50], Val Losses: mse: 2.6

##### do not use ablations: rotations (8,4)

In [4]:
importlib.reload(monotonic)
importlib.reload(train_config)

cfg = train_config.FlatACLConfig( 
    seq_len=336,
    pred_len=336,
    channels=data_mgr.datasets['rossler']['channels'],# data_mgr.channels,              # ← number of features in your data
    batch_size=128,
    learning_rate=9e-4, 
    seeds=[1955, 7, 20],  
    epochs=50, 
    dim_hidden=128,
    dim_augment=128,
    ablate_no_koopman=False,
    use_complex_eigenvalues=True,
    second_delay_use_shift=True,
    ablate_rotate_back_Koopman=False, 
    ablate_shift_inside_scale=False,
)
cfg.x_to_z_delay.enable_magnitudes_scale_shift = [False, True]
cfg.x_to_z_deri.enable_magnitudes_scale_shift = [False, True]
cfg.z_to_x_main.enable_magnitudes_scale_shift = [False, True]
cfg.z_to_y_main.enable_magnitudes_scale_shift = [False, True]
exp = execute_model_evaluation('rossler', cfg, data_mgr, scale=False)

Shape of training data: torch.Size([36400, 3])
Shape of validation data: torch.Size([5200, 3])
Shape of testing data: torch.Size([10400, 3])
global_std.shape: torch.Size([3])
Global Std for rossler: tensor([5.0964, 4.7939, 2.8266], device='cuda:0')
Train set sample shapes: torch.Size([336, 3]), torch.Size([336, 3])
Validation set sample shapes: torch.Size([336, 3]), torch.Size([336, 3])
Test set data shapes: torch.Size([10400, 3]), torch.Size([10400, 3])
Number of batches in train_loader: 280
Batch 0: Data shape torch.Size([128, 336, 3]), Target shape torch.Size([128, 336, 3])

Data Preparation: rossler
Sequence Length: 336
Prediction Length: 336
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 280
Validation Batches: 36
Test Batches: 77

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 3.2017, mae: 0.6333, huber: 0.4076, swd: 2.6055, ept: 300.8536
Epoch [1/50], Val Losses: mse: 1.1

#### pred=720

##### huber

In [14]:
from monotonic import DynamicTanh
import torch.nn as nn

importlib.reload(monotonic)
importlib.reload(train_config) 
cfg = train_config.FlatACLConfig(  # original householder 
    seq_len=336,
    pred_len=720,
    channels=data_mgr.datasets['rossler']['channels'],# data_mgr.channels,              # ← number of features in your data
    batch_size=128,
    learning_rate=9e-4, 
    seeds=[1955, 7, 20],  
    epochs=50, 
    dim_hidden=128,
    dim_augment=128, 
    ablate_no_koopman=False,
    use_complex_eigenvalues=True,
    second_delay_use_shift=True,
    ablate_rotate_back_Koopman=True, 
    ablate_shift_inside_scale=False,
    householder_reflects_latent = 2,
    householder_reflects_data = 4,
    mixing_strategy='delay_only', 
    loss_backward_weights = [0.0, 0.0, 1.0, 0.0, 0.0],
    loss_validate_weights = [0.0, 0.0, 1.0, 0.0, 0.0],
    ablate_deterministic_y0=False, 
)
cfg.x_to_z_delay.enable_magnitudes = [False, True]
cfg.x_to_z_delay.spectral_flags_scale_shift = [True, False]
cfg.x_to_z_delay.spectral_flags_magnitudes = [False, True]
cfg.x_to_z_delay.spectral_flags_hidden_layers = [False, False]
cfg.x_to_z_delay.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.x_to_z_delay.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]

cfg.x_to_z_deri.enable_magnitudes = [False, True]
cfg.x_to_z_deri.spectral_flags_scale_shift = [True, False]
cfg.x_to_z_deri.spectral_flags_magnitudes = [False, True]
cfg.x_to_z_deri.spectral_flags_hidden_layers = [False, False]
cfg.x_to_z_deri.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.x_to_z_deri.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]

cfg.z_to_x_main.enable_magnitudes = [False, True]
cfg.z_to_x_main.spectral_flags_scale_shift = [True, False]
cfg.z_to_x_main.spectral_flags_magnitudes = [False, True]
cfg.z_to_x_main.spectral_flags_hidden_layers = [False, False]
cfg.z_to_x_main.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_to_x_main.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]

cfg.z_push_to_z.enable_magnitudes = [False, True]
cfg.z_push_to_z.spectral_flags_scale_shift = [True, False]
cfg.z_push_to_z.spectral_flags_magnitudes = [False, True]
cfg.z_push_to_z.spectral_flags_hidden_layers = [False, False]
cfg.z_push_to_z.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_push_to_z.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]

cfg.z_to_y_main.enable_magnitudes = [False, True]
cfg.z_to_y_main.spectral_flags_scale_shift = [True, False]
cfg.z_to_y_main.spectral_flags_magnitudes = [False, True]
cfg.z_to_y_main.spectral_flags_hidden_layers = [False, False]
cfg.z_to_y_main.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_to_y_main.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]
exp = execute_model_evaluation('rossler', cfg, data_mgr, scale=False)

Shape of training data: torch.Size([36400, 3])
Shape of validation data: torch.Size([5200, 3])
Shape of testing data: torch.Size([10400, 3])
global_std.shape: torch.Size([3])
Global Std for rossler: tensor([5.0964, 4.7939, 2.8266], device='cuda:0')
Train set sample shapes: torch.Size([336, 3]), torch.Size([720, 3])
Validation set sample shapes: torch.Size([336, 3]), torch.Size([720, 3])
Test set data shapes: torch.Size([10400, 3]), torch.Size([10400, 3])
Number of batches in train_loader: 277
Batch 0: Data shape torch.Size([128, 336, 3]), Target shape torch.Size([128, 720, 3])

Data Preparation: rossler
Sequence Length: 336
Prediction Length: 720
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 277
Validation Batches: 33
Test Batches: 74

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 4.3458, mae: 0.8665, huber: 0.6013, swd: 3.2119, ept: 562.3547
Epoch [1/50], Val Losses: mse: 3.3

### Timemixer

#### pred=96

##### huber

In [9]:
utils.reload_modules([utils])
cfg = train_config.FlatTimeMixerConfig(
    seq_len=336,
    pred_len=96,
    channels=data_mgr.datasets['rossler']['channels'],
    enc_in=data_mgr.datasets['rossler']['channels'],
    dec_in=data_mgr.datasets['rossler']['channels'],
    c_out=data_mgr.datasets['rossler']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50, 
)
exp = execute_model_evaluation('rossler', cfg, data_mgr, scale=False)


Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([36400, 3])
Shape of validation data: torch.Size([5200, 3])
Shape of testing data: torch.Size([10400, 3])
global_std.shape: torch.Size([3])
Global Std for rossler: tensor([5.0964, 4.7939, 2.8266], device='cuda:0')
Train set sample shapes: torch.Size([336, 3]), torch.Size([96, 3])
Validation set sample shapes: torch.Size([336, 3]), torch.Size([96, 3])
Test set data shapes: torch.Size([10400, 3]), torch.Size([10400, 3])
Number of batches in train_loader: 282
Batch 0: Data shape torch.Size([128, 336, 3]), Target shape torch.Size([128, 96, 3])

Data Preparation: rossler
Sequence Length: 336
Prediction Length: 96
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 282
Validation Batches: 38
Test Batches: 78

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 4.7929, mae: 1.0037, huber: 0.7032, sw

#### pred=196

##### huber

In [10]:
utils.reload_modules([utils])
cfg = train_config.FlatTimeMixerConfig(
    seq_len=336,
    pred_len=196,
    channels=data_mgr.datasets['rossler']['channels'],
    enc_in=data_mgr.datasets['rossler']['channels'],
    dec_in=data_mgr.datasets['rossler']['channels'],
    c_out=data_mgr.datasets['rossler']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50, 
)
exp = execute_model_evaluation('rossler', cfg, data_mgr, scale=False)


Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([36400, 3])
Shape of validation data: torch.Size([5200, 3])
Shape of testing data: torch.Size([10400, 3])
global_std.shape: torch.Size([3])
Global Std for rossler: tensor([5.0964, 4.7939, 2.8266], device='cuda:0')
Train set sample shapes: torch.Size([336, 3]), torch.Size([196, 3])
Validation set sample shapes: torch.Size([336, 3]), torch.Size([196, 3])
Test set data shapes: torch.Size([10400, 3]), torch.Size([10400, 3])
Number of batches in train_loader: 281
Batch 0: Data shape torch.Size([128, 336, 3]), Target shape torch.Size([128, 196, 3])

Data Preparation: rossler
Sequence Length: 336
Prediction Length: 196
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 281
Validation Batches: 37
Test Batches: 78

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 6.1355, mae: 1.2860, huber: 0.9515

#### pred=336

##### huber

In [5]:
utils.reload_modules([utils])
cfg = train_config.FlatTimeMixerConfig(
    seq_len=336,
    pred_len=336,
    channels=data_mgr.datasets['rossler']['channels'],
    enc_in=data_mgr.datasets['rossler']['channels'],
    dec_in=data_mgr.datasets['rossler']['channels'],
    c_out=data_mgr.datasets['rossler']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50, 
)
exp = execute_model_evaluation('rossler', cfg, data_mgr, scale=False)


Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([36400, 3])
Shape of validation data: torch.Size([5200, 3])
Shape of testing data: torch.Size([10400, 3])
global_std.shape: torch.Size([3])
Global Std for rossler: tensor([5.0964, 4.7939, 2.8266], device='cuda:0')
Train set sample shapes: torch.Size([336, 3]), torch.Size([336, 3])
Validation set sample shapes: torch.Size([336, 3]), torch.Size([336, 3])
Test set data shapes: torch.Size([10400, 3]), torch.Size([10400, 3])
Number of batches in train_loader: 280
Batch 0: Data shape torch.Size([128, 336, 3]), Target shape torch.Size([128, 336, 3])

Data Preparation: rossler
Sequence Length: 336
Prediction Length: 336
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 280
Validation Batches: 36
Test Batches: 77

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 7.0310, mae: 1.4236, huber: 1.0840

#### pred=720

##### huber

In [11]:
utils.reload_modules([utils])
cfg = train_config.FlatTimeMixerConfig(
    seq_len=336,
    pred_len=720,
    channels=data_mgr.datasets['rossler']['channels'],
    enc_in=data_mgr.datasets['rossler']['channels'],
    dec_in=data_mgr.datasets['rossler']['channels'],
    c_out=data_mgr.datasets['rossler']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50, 
)
exp = execute_model_evaluation('rossler', cfg, data_mgr, scale=False)


Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([36400, 3])
Shape of validation data: torch.Size([5200, 3])
Shape of testing data: torch.Size([10400, 3])
global_std.shape: torch.Size([3])
Global Std for rossler: tensor([5.0964, 4.7939, 2.8266], device='cuda:0')
Train set sample shapes: torch.Size([336, 3]), torch.Size([720, 3])
Validation set sample shapes: torch.Size([336, 3]), torch.Size([720, 3])
Test set data shapes: torch.Size([10400, 3]), torch.Size([10400, 3])
Number of batches in train_loader: 277
Batch 0: Data shape torch.Size([128, 336, 3]), Target shape torch.Size([128, 720, 3])

Data Preparation: rossler
Sequence Length: 336
Prediction Length: 720
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 277
Validation Batches: 33
Test Batches: 74

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 8.8553, mae: 1.7670, huber: 1.4100

### PatchTST

#### pred=96

##### huber

In [3]:
utils.reload_modules([utils])
cfg = train_config.FlatPatchTSTConfig(
    seq_len=336,
    pred_len=96,
    channels=data_mgr.datasets['rossler']['channels'],
    enc_in=data_mgr.datasets['rossler']['channels'],
    dec_in=data_mgr.datasets['rossler']['channels'],
    c_out=data_mgr.datasets['rossler']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50,
    task_name='long_term_forecast',
    factor=3,
)
exp = execute_model_evaluation('rossler', cfg, data_mgr, scale=False)

Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([36400, 3])
Shape of validation data: torch.Size([5200, 3])
Shape of testing data: torch.Size([10400, 3])
global_std.shape: torch.Size([3])
Global Std for rossler: tensor([5.0964, 4.7939, 2.8266], device='cuda:0')
Train set sample shapes: torch.Size([336, 3]), torch.Size([96, 3])
Validation set sample shapes: torch.Size([336, 3]), torch.Size([96, 3])
Test set data shapes: torch.Size([10400, 3]), torch.Size([10400, 3])
Number of batches in train_loader: 282
Batch 0: Data shape torch.Size([128, 336, 3]), Target shape torch.Size([128, 96, 3])

Data Preparation: rossler
Sequence Length: 336
Prediction Length: 96
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 282
Validation Batches: 38
Test Batches: 78

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 7.3591, mae: 1.4447, huber: 1.1071, sw

#### pred=196

##### huber

In [4]:
utils.reload_modules([utils])
cfg = train_config.FlatPatchTSTConfig(
    seq_len=336,
    pred_len=196,
    channels=data_mgr.datasets['rossler']['channels'],
    enc_in=data_mgr.datasets['rossler']['channels'],
    dec_in=data_mgr.datasets['rossler']['channels'],
    c_out=data_mgr.datasets['rossler']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50,
    task_name='long_term_forecast',
    factor=3,
)
exp = execute_model_evaluation('rossler', cfg, data_mgr, scale=False)

Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([36400, 3])
Shape of validation data: torch.Size([5200, 3])
Shape of testing data: torch.Size([10400, 3])
global_std.shape: torch.Size([3])
Global Std for rossler: tensor([5.0964, 4.7939, 2.8266], device='cuda:0')
Train set sample shapes: torch.Size([336, 3]), torch.Size([196, 3])
Validation set sample shapes: torch.Size([336, 3]), torch.Size([196, 3])
Test set data shapes: torch.Size([10400, 3]), torch.Size([10400, 3])
Number of batches in train_loader: 281
Batch 0: Data shape torch.Size([128, 336, 3]), Target shape torch.Size([128, 196, 3])

Data Preparation: rossler
Sequence Length: 336
Prediction Length: 196
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 281
Validation Batches: 37
Test Batches: 78

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 9.5338, mae: 1.7317, huber: 1.3725

#### pred=336

##### huber

In [6]:
utils.reload_modules([utils])
cfg = train_config.FlatPatchTSTConfig(
    seq_len=336,
    pred_len=336,
    channels=data_mgr.datasets['rossler']['channels'],
    enc_in=data_mgr.datasets['rossler']['channels'],
    dec_in=data_mgr.datasets['rossler']['channels'],
    c_out=data_mgr.datasets['rossler']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50,
    task_name='long_term_forecast',
    factor=3,
)
exp = execute_model_evaluation('rossler', cfg, data_mgr, scale=False)

Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([36400, 3])
Shape of validation data: torch.Size([5200, 3])
Shape of testing data: torch.Size([10400, 3])
global_std.shape: torch.Size([3])
Global Std for rossler: tensor([5.0964, 4.7939, 2.8266], device='cuda:0')
Train set sample shapes: torch.Size([336, 3]), torch.Size([336, 3])
Validation set sample shapes: torch.Size([336, 3]), torch.Size([336, 3])
Test set data shapes: torch.Size([10400, 3]), torch.Size([10400, 3])
Number of batches in train_loader: 280
Batch 0: Data shape torch.Size([128, 336, 3]), Target shape torch.Size([128, 336, 3])

Data Preparation: rossler
Sequence Length: 336
Prediction Length: 336
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 280
Validation Batches: 36
Test Batches: 77

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 10.5670, mae: 1.9265, huber: 1.556

#### pred=720

##### huber

In [5]:
utils.reload_modules([utils])
cfg = train_config.FlatPatchTSTConfig(
    seq_len=336,
    pred_len=720,
    channels=data_mgr.datasets['rossler']['channels'],
    enc_in=data_mgr.datasets['rossler']['channels'],
    dec_in=data_mgr.datasets['rossler']['channels'],
    c_out=data_mgr.datasets['rossler']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50,
    task_name='long_term_forecast',
    factor=3,
)
exp = execute_model_evaluation('rossler', cfg, data_mgr, scale=False)

Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([36400, 3])
Shape of validation data: torch.Size([5200, 3])
Shape of testing data: torch.Size([10400, 3])
global_std.shape: torch.Size([3])
Global Std for rossler: tensor([5.0964, 4.7939, 2.8266], device='cuda:0')
Train set sample shapes: torch.Size([336, 3]), torch.Size([720, 3])
Validation set sample shapes: torch.Size([336, 3]), torch.Size([720, 3])
Test set data shapes: torch.Size([10400, 3]), torch.Size([10400, 3])
Number of batches in train_loader: 277
Batch 0: Data shape torch.Size([128, 336, 3]), Target shape torch.Size([128, 720, 3])

Data Preparation: rossler
Sequence Length: 336
Prediction Length: 720
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 277
Validation Batches: 33
Test Batches: 74

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 11.5939, mae: 2.1447, huber: 1.766

### DLinear

#### pred=96

##### huber

In [7]:
importlib.reload(monotonic)
importlib.reload(train_config)
utils.reload_modules([utils])
cfg = train_config.FlatDLinearConfig(
    seq_len=336,
    pred_len=96,
    channels=data_mgr.datasets['rossler']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50, 
)
exp = execute_model_evaluation('rossler', cfg, data_mgr, scale=False)

Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([36400, 3])
Shape of validation data: torch.Size([5200, 3])
Shape of testing data: torch.Size([10400, 3])
global_std.shape: torch.Size([3])
Global Std for rossler: tensor([5.0964, 4.7939, 2.8266], device='cuda:0')
Train set sample shapes: torch.Size([336, 3]), torch.Size([96, 3])
Validation set sample shapes: torch.Size([336, 3]), torch.Size([96, 3])
Test set data shapes: torch.Size([10400, 3]), torch.Size([10400, 3])
Number of batches in train_loader: 282
Batch 0: Data shape torch.Size([128, 336, 3]), Target shape torch.Size([128, 96, 3])

Data Preparation: rossler
Sequence Length: 336
Prediction Length: 96
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 282
Validation Batches: 38
Test Batches: 78

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 3.7384, mae: 0.7961, huber: 0.5433, sw

#### pred=196

##### huber

In [8]:
importlib.reload(monotonic)
importlib.reload(train_config)
utils.reload_modules([utils])
cfg = train_config.FlatDLinearConfig(
    seq_len=336,
    pred_len=196,
    channels=data_mgr.datasets['rossler']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50, 
)
exp = execute_model_evaluation('rossler', cfg, data_mgr, scale=False)

Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([36400, 3])
Shape of validation data: torch.Size([5200, 3])
Shape of testing data: torch.Size([10400, 3])
global_std.shape: torch.Size([3])
Global Std for rossler: tensor([5.0964, 4.7939, 2.8266], device='cuda:0')
Train set sample shapes: torch.Size([336, 3]), torch.Size([196, 3])
Validation set sample shapes: torch.Size([336, 3]), torch.Size([196, 3])
Test set data shapes: torch.Size([10400, 3]), torch.Size([10400, 3])
Number of batches in train_loader: 281
Batch 0: Data shape torch.Size([128, 336, 3]), Target shape torch.Size([128, 196, 3])

Data Preparation: rossler
Sequence Length: 336
Prediction Length: 196
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 281
Validation Batches: 37
Test Batches: 78

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 5.5777, mae: 1.0754, huber: 0.7866

#### pred=336

##### huber

In [7]:
importlib.reload(monotonic)
importlib.reload(train_config)
utils.reload_modules([utils])
cfg = train_config.FlatDLinearConfig(
    seq_len=336,
    pred_len=336,
    channels=data_mgr.datasets['rossler']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50, 
)
exp = execute_model_evaluation('rossler', cfg, data_mgr, scale=False)

Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([36400, 3])
Shape of validation data: torch.Size([5200, 3])
Shape of testing data: torch.Size([10400, 3])
global_std.shape: torch.Size([3])
Global Std for rossler: tensor([5.0964, 4.7939, 2.8266], device='cuda:0')
Train set sample shapes: torch.Size([336, 3]), torch.Size([336, 3])
Validation set sample shapes: torch.Size([336, 3]), torch.Size([336, 3])
Test set data shapes: torch.Size([10400, 3]), torch.Size([10400, 3])
Number of batches in train_loader: 280
Batch 0: Data shape torch.Size([128, 336, 3]), Target shape torch.Size([128, 336, 3])

Data Preparation: rossler
Sequence Length: 336
Prediction Length: 336
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 280
Validation Batches: 36
Test Batches: 77

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 6.8865, mae: 1.2651, huber: 0.9547

#### pred=720

##### huber

In [6]:
importlib.reload(monotonic)
importlib.reload(train_config)
utils.reload_modules([utils])
cfg = train_config.FlatDLinearConfig(
    seq_len=336,
    pred_len=720,
    channels=data_mgr.datasets['rossler']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50, 
)
exp = execute_model_evaluation('rossler', cfg, data_mgr, scale=False)

Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([36400, 3])
Shape of validation data: torch.Size([5200, 3])
Shape of testing data: torch.Size([10400, 3])
global_std.shape: torch.Size([3])
Global Std for rossler: tensor([5.0964, 4.7939, 2.8266], device='cuda:0')
Train set sample shapes: torch.Size([336, 3]), torch.Size([720, 3])
Validation set sample shapes: torch.Size([336, 3]), torch.Size([720, 3])
Test set data shapes: torch.Size([10400, 3]), torch.Size([10400, 3])
Number of batches in train_loader: 277
Batch 0: Data shape torch.Size([128, 336, 3]), Target shape torch.Size([128, 720, 3])

Data Preparation: rossler
Sequence Length: 336
Prediction Length: 720
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 277
Validation Batches: 33
Test Batches: 74

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 8.1739, mae: 1.6156, huber: 1.2781